# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [27]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
city_data_df = pd.read_csv('../output_data/cities.csv')
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,37.06,75,0,4.61,AR,1624756183
1,1,ola,59.5833,151.2833,59.72,78,40,7.05,RU,1624755636
2,2,bluff,-46.6000,168.3333,51.80,87,89,13.80,NZ,1624756183
3,3,kapaa,22.0752,-159.3190,85.98,73,90,3.00,US,1624756183
4,4,mount gambier,-37.8333,140.7667,55.08,82,27,3.24,AU,1624756184


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
gmaps.configure(api_key=g_key)

In [30]:
locations = city_data_df[['Lat','Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
filtered_cities_df = city_data_df.loc[(city_data_df['Max Temp'] < 80) & (city_data_df['Max Temp'] > 70) 
                                      & (city_data_df['Wind Speed'] < 10) 
                                      & (city_data_df['Cloudiness'] == 0)].dropna()
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
124,124,cayenne,4.9333,-52.3333,79.09,86,0,1.01,GF,1624756219
155,155,olinda,-8.0089,-34.8553,71.33,73,0,9.22,BR,1624755767
157,157,santarem,-2.4431,-54.7083,75.33,100,0,0.00,BR,1624756228
193,193,mao,39.8885,4.2658,70.36,83,0,3.44,ES,1624755644
262,262,mozarlandia,-14.7447,-50.5706,72.84,25,0,6.15,BR,1624756259
265,265,aguilas,37.4063,-1.5829,70.05,69,0,4.21,ES,1624756260
277,277,gat,31.6100,34.7642,76.41,77,0,1.92,IL,1624756202
390,390,sibenik,43.7272,15.9058,73.06,61,0,5.57,HR,1624756298
499,499,aksu,41.1231,80.2644,77.16,28,0,4.81,CN,1624756330


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
124,cayenne,GF,4.9333,-52.3333,
155,olinda,BR,-8.0089,-34.8553,
157,santarem,BR,-2.4431,-54.7083,
193,mao,ES,39.8885,4.2658,
262,mozarlandia,BR,-14.7447,-50.5706,
265,aguilas,ES,37.4063,-1.5829,
277,gat,IL,31.6100,34.7642,
390,sibenik,HR,43.7272,15.9058,
499,aksu,CN,41.1231,80.2644,


In [33]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
124,cayenne,GF,4.9333,-52.3333,
155,olinda,BR,-8.0089,-34.8553,
157,santarem,BR,-2.4431,-54.7083,
193,mao,ES,39.8885,4.2658,
262,mozarlandia,BR,-14.7447,-50.5706,
265,aguilas,ES,37.4063,-1.5829,
277,gat,IL,31.6100,34.7642,
390,sibenik,HR,43.7272,15.9058,
499,aksu,CN,41.1231,80.2644,


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))